In [2]:
# -*- coding: <utf-8> -*-
import wfdb
import matplotlib.pyplot as plt
import numpy as np
from hrv.filters import quotient, moving_median
from scipy import interpolate
from tqdm import tqdm
FS = 100.0

In [5]:
def time_rr_interval(rri):
    #convert time to seconds
    rri_time = np.cumsum(rri) / 1000.0
    #time should start from zero
    return rri_time - rri_time[0]  

def create_interp_time(rri, fs):
    time_rri = time_rr_interval(rri)
    return np.arange(0, time_rri[-1], 1 / float(fs))

def time_cubic_spline_qrs(qrs_index, qrs_amp, fs):
    time_qrs = qrs_index / float(FS)
    time_qrs = time_qrs - time_qrs[0]
    time_qrs_interp = np.arange(0, time_qrs[-1], 1/float(fs))
    tck = interpolate.splrep(time_qrs, qrs_amp, s=0)
    qrs_interp = interpolate.splev(time_qrs_interp, tck, der=0)
    return time_qrs_interp, qrs_interp

def time_cubic_spline_rri(rri, fs):
    time_rri = time_rr_interval(rri)
    time_rri_interp = create_interp_time(rri, fs)
    tck = interpolate.splrep(time_rri, rri, s=0)
    rri_interp = interpolate.splev(time_rri_interp, tck, der=0)
    return time_rri_interp, rri_interp


In [ ]:
data_path= 'C:/Users/HP/PROJECT1/www.physionet.org/physiobank/database/apnea-ecg/'
train_data_name = ['a01', 'a02', 'a03', 'a04', 'a05',
             'a06', 'a07', 'a08', 'a09', 'a10',
             'a11', 'a12', 'a13', 'a14', 'a15',
             'a16', 'a17', 'a18', 'a19', 'a20',
             'b01', 'b02', 'b03', 'b04', 'b05',
             'c01', 'c02', 'c03', 'c04', 'c05',
             'c06', 'c07', 'c08', 'c09', 'c10'
             ]
age = [51, 38, 54, 52, 58,
       63, 44, 51, 52, 58,
       58, 52, 51, 51, 60,
       44, 40, 52, 55, 58,
       44, 53, 53, 42, 52,
       31, 37, 39, 41, 28,
       28, 30, 42, 37, 27]
gender = [1, 1, 1, 1, 1,
       1, 1, 1, 1, 1,
       1, 1, 1, 1, 1,
       1, 1, 1, 1, 1,
       0, 1, 1, 1, 1,
       1, 1, 1, 0, 0,
       0, 0, 1, 1, 1]


In [ ]:
def get_qrs_amp(ecg, qrs):
    interval = int(FS * 0.250)
    qrs_amp = []
    for index in range(len(qrs)):
        curr_qrs = qrs[index]
        amp = np.max(ecg[curr_qrs-interval:curr_qrs+interval])
        qrs_amp.append(amp)

    return qrs_amp

In [ ]:
MARGIN = 10
FS_INTP = 4
MAX_HR = 300.0
MIN_HR = 20.0
MIN_RRI = 1.0 / (MAX_HR / 60.0) * 1000
MAX_RRI = 1.0 / (MIN_HR / 60.0) * 1000
input_array = []
label_array = []
for data_index in range(len(train_data_name)):
    print (train_data_name[data_index])
    win_num = len(wfdb.rdann(data_path + train_data_name[data_index], 'apn').symbol)
    signals, fields = wfdb.rdsamp(data_path + train_data_name[data_index])
    for index in tqdm(range(1, win_num)):
        samp_from = index * 60 * FS # segmented to 60 seconds
        samp_to = samp_from + 60 * FS  # segmented to 60 seconds

        qrs_ann = wfdb.rdann(data_path + train_data_name[data_index], 'qrs', sampfrom=samp_from - (MARGIN*100), sampto=samp_to + (MARGIN*100)).sample
        apn_ann = wfdb.rdann(data_path + train_data_name[data_index], 'apn', sampfrom=samp_from, sampto=samp_to-1).symbol

        qrs_amp = get_qrs_amp(signals, qrs_ann)

        rri = np.diff(qrs_ann)
        rri_ms = rri.astype('float') / FS * 1000.0

        rri_filter = moving_median(rri_ms)
        qrs_filter = moving_median(qrs_amp)
        if rri_filter.shape[0] > 5 and (np.min(rri_filter) >= MIN_RRI and np.max(rri_filter) <= MAX_RRI):
            time_intp, rri_intp = time_cubic_spline_rri(rri_filter, FS_INTP)
            qrs_time_intp, qrs_intp = time_cubic_spline_qrs(qrs_ann, qrs_amp, FS_INTP)
            rri_intp = rri_intp[(time_intp >= MARGIN) & (time_intp < (60+MARGIN))]
            qrs_intp = qrs_intp[(qrs_time_intp >= MARGIN) & (qrs_time_intp < (60 + MARGIN))]

            if len(rri_intp) != (FS_INTP * 60):
                flag = 1
            else:
                flag = 0

            if flag == 0:
                rri_intp = rri_intp - np.mean(rri_intp)
                qrs_intp = qrs_intp - np.mean(qrs_intp)
                # N denotes Normal and A denotes Apnea
                if apn_ann[0] == 'N':
                    label = 0.0
                elif apn_ann[0] == 'A':
                    label = 1.0
                else:
                    label = 2.0

                input_array.append([rri_intp, qrs_intp, age[data_index], gender[data_index]])
                label_array.append(label)
        else:
            xyz=1

np.save('train_input.npy', input_array)
np.save('train_label.npy', label_array)

a01


100%|████████████████████████████████████████████████████████████████████████████████| 488/488 [07:48<00:00,  1.78s/it]


a02


100%|████████████████████████████████████████████████████████████████████████████████| 527/527 [09:33<00:00,  2.06s/it]


a03


...

100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [09:46<00:00,  3.07s/it]


a06


100%|████████████████████████████████████████████████████████████████████████████████| 509/509 [11:04<00:00,  2.39s/it]


a07


 19%|███████████████▋                                                                 | 99/510 [00:35<03:19,  2.06it/s]